In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../data/final/master_dataset.csv")
df["dateTime"] = pd.to_datetime(df["dateTime"])
df = df.sort_values("dateTime")
df.head()


,dateTime,TN,TP,NH3,NO23,OP,SSC
0,1978-06-05,2.51,0.05,0.02,1.8,0.05,247.0
1,1978-07-12,2.13,0.19,0.02,1.3,0.05,359.0
2,1978-08-15,2.47,0.34,0.17,1.5,0.05,367.0
3,1978-09-14,2.40,0.16,0.02,0.6,0.05,159.0
4,1978-10-23,1.98,0.20,0.02,1.3,0.05,196.0


In [3]:
tn_series = df["TN"].values


In [4]:
def create_sequences(data, window_size=5):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

X, y = create_sequences(tn_series, window_size=5)

print(X.shape, y.shape)


(536, 5) (536,)


In [6]:
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)


(536, 5, 1)


In [5]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [8]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test  = X_test.reshape(X_test.shape[0],  X_test.shape[1], 1)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),

    LSTM(32),
    Dropout(0.2),

    Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse"
)

model.summary()


c:\Users\sagam\miniconda3\envs\aquasense\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
c:\Users\sagam\miniconda3\envs\aquasense\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 64)          │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,345 (114.63 KB)

 Trainable params: 29,345 (114.63 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_test, y_test)
)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - loss: 1.8259 - val_loss: 0.4616
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5478 - val_loss: 0.2667
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5049 - val_loss: 0.2623
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4877 - val_loss: 0.2686
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.4943 - val_loss: 0.2633
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.4840 - val_loss: 0.2619
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.4714 - val_loss: 0.2550
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4766 - val_loss: 0.2388
Epoch 9/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4434 - val_loss: 0.2381
Epoch 10/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.4075 - val_loss: 0.2239
Epoch 11/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.3876 - val_loss: 0.2374
Epoch 12/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.3

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("LSTM RMSE:", rmse)
print("LSTM R2:", r2)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
LSTM RMSE: 0.43427792535225584
LSTM R2: 0.35324435257046116
